In [6]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
import seaborn as sns
import pickle

In [ ]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

In [6]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [7]:
#df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
#df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

#df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
#df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

#df = df[(df['duration'] >= 1) & (df['duration'] <= 60)] 

#df

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
df[categorical] = df[categorical].astype(str)

In [ ]:
#train_dicts = df[categorical + numerical].to_dict(orient = 'records')

In [ ]:
#dv = DictVectorizer()
#X_train = dv.fit_transform(train_dicts)

In [8]:
#sns.displot(y_pred, label = 'prediction')
#sns.displot(y_train, label = 'actual values')

#plt.legend()

In [ ]:
def read_dataframe(filename): 
    df = pd.read_csv(filename)

    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)] 

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df
    

In [ ]:
df_train = read_dataframe('link or filename to the train data')
df_val = read_dataframe('link or filename to the validation data')

In [ ]:
len(df_train), len(df_val)

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse = mean_squared_error(y_val, y_pred, squared = False)
rmse

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
with mlflow.start_run(): 
   
   mlflow.set_tag("developer", "MaxTheKing")

   mlflow.log_param("train-data-path", "./data/green_tripdata_2023-01.csv")
   mlflow.log_param("test-data-path", "./data/green_tripdata_2023-02.csv")

   alpha = 0.01
   mlflow.log_param("alpha", alpha)

   lr = Lasso(alpha)
   lr.fit(X_train, y_train)

   y_pred = lr.predict(X_val)

   rmse = mean_squared_error(y_val, y_pred, squared = False)
   mlflow.log_metric("rmse", rmse)

   mlflow.log_artifact(local_path = "models/lin_re.bin", artifact_path= "models_pickle/")

In [ ]:
import xgboost as xgb 

train = xgb.DMatrix(X_train, label = y_train)
valid = xgb.DMatrix(X_val, label = y_val)

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


def objective(params): 

    with mlflow.start_run(): 
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params = params, 
            dtrain = train, 
            num_boost_round = 1000, 
            evals = [(valid, "validation")], 
            early_stopping_rounds = 50
        )

        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared = False)
        mlflow.log_metric("rmse", rmse)

    return {'loss' : rmse, 'status': STATUS_OK }



search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)), 
    'learning_rate': hp.loguniform('learning_rate', -3, 0), 
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1), 
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1) ,
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3), 
    'objective': 'reg;linear', 
    'seed': 42
}


best_result = fmin(
    fn = objective, 
    space = search_space, 
    algo = tpe.suggest, 
    max_evals = 50, 
    trials = Trials()
)

In [ ]:
with mlflow.start_run(): 

    train = xgb.DMatrix(X_train, label = y_train)
    valid = xgb.DMatrix(X_val, label = y_val)



    best_params = {
    'learning_rate': 0.20472169880371677, 
    'max_depth': 17,
    'min_child_weight' : 1.240261172004385,
    'objective': 'reg:linear', 
    'reg_alpha': 0.28567896734700793, 
    'reg_lambda': 0.004264404814393109,
    'seed': 42 

      }

#mlflow.xgboost.autolog()
    mlflow.log_params(best_params)

    booster = xgb.train(
    params = best_params, 
    dtrain = train, 
    num_boost_round = 1000, 
    evals = [(valid, "validation")]
    early_stopping_rounds = 50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared = False)
    mlflow.log_metric("rmse", rmse)

    with open('models/preprocessor.b', 'wb') as f_out: 
       pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path = "preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path = "models_mlflow")









